### Propensity Scores (PS)

PS offer a solution to mimic a randomized controlled trial (RCT) using observational data.

In an RCT, we’d randomly assign some pixels to be treated and others to be control. Because of randomization, the two groups would be identical on average in all other characteristics (same soil, same elevation, same prior climate).
In ESS, we can’t do this. For example, the pixels that were reforested were chosen for a reason (e.g., policy, good soil, near a road). This creates selection bias.

Matching methods are designed to correct this selection bias by selecting samples to find similar comparisons. However, we will use PS to manually discard extreme samples that have a very high or very low probability of being treated using logistic regression. We wont use matching methods.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WinterSchool2026/ch09-causal-inference-extremes/blob/main/notebooks/02_data_filtering_propensity_score.ipynb)

In [ ]:
# Upgrade pip first for better dependency resolution
!pip install -U pip

In [ ]:
# Install packages, ensuring numpy is at a version compatible with most 2024-2025 builds
!pip install -q econml numba xarray zarr fsspec aiohttp geopandas dask netcdf4 h5netcdf "numpy<2.0"

Filters data to find the best examples for comparison. A propensity score is a number for each sample that answers the question: Given all the confounders for this sample, what was the probability that it would receive the treatment? It's a propensity or tendency to be treated.

In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sys
import os
from google.colab import drive


Mount the folder with the utils functions

In [ ]:
# 1. Mount drive if you haven't already
drive.mount('/content/drive')

In [ ]:
# 2. Append the PARENT directory (notebooks), not the utils folder itself
path_to_parent = '/content/drive/MyDrive/09_challenge_EllisWinterSchool'
if path_to_parent not in sys.path:
    sys.path.append(path_to_parent)

# 3. Now Python sees 'utils' as a package inside 'notebooks'
import utils.utils
from utils.utils import *

print("✅ Success! Functions imported.")

Read the sample points:

In [ ]:
samples = pd.read_csv('/content/drive/MyDrive/09_challenge_EllisWinterSchool/dataset_clim_env_oci_norm.csv')

In [ ]:
bound_country = gpd.read_file('/content/drive/MyDrive/09_challenge_EllisWinterSchool/world-administrative-boundaries.geojson')

## Train a logisitic regression 

For PS we will train a classifier that uses the control variables to predict the Treatment (T).

- Step 1: train a classifier, logistic regression, to predict the treatment (T) using confounders (W), the output is the PS for each sample.
- Step 2: Filter. Filter out samples that have propensity score above *0.90/0.95* which obvious it would be treated, and below *0.1/0.05* very unlikely to be treated.

This step has to be done using the selected factors (confounders). If we change the confounders the PS has to be done again.

In [ ]:
inp_vars = ['E_gleam_ds','S_gleam_ds','H_gleam_ds',
            'pev_ds','sro_ds','sp_ds','tp_ds','d2m_ds',
            'agri_irri', 'agri_mix', 'agri_rain',
            'soil_clay', 'soil_oc', 'soil_roots','soil_sand', 'soil_tawc',
            'lst_night_ds','ndvi_ds','ndwi_ds',
            'pop','road','hand','lc2','lc3','lc5','lc8',
            'censo','soi_long','pdo_timeseries_sstens','noaa_globaltmp_comb']

In [ ]:
treatment = "SMA"

In [ ]:
# create the TREATMENTS
samples['SMA_2'] = np.where(samples['SMA'] >= 2, 1, 0)
samples = samples.drop(treatment, axis=1)

In [ ]:
treatment = "SMA_2"

In [ ]:
fig, axs = plot_spatial_temporal_grid(samples, treatment, bound_country)
plt.show()

---

## Propensity Score

In this example, we use Soil Moisture Anomaly (SMA) as a treatment (T), where SMA >= 2 is treated and < 2 non-treated 

In [ ]:
ps_vars = ["DI_agri_extreme_M7","id", treatment]  + inp_vars

In [ ]:
df_ps = samples[ps_vars].copy()

In [ ]:
df_ps.head()

Check the means for control and treatment

In [ ]:
df_ps.groupby(treatment).mean(numeric_only=True)

Separate control and treatment for t-test

In [ ]:
df_control = df_ps[df_ps[treatment] == 0]
df_treatment = df_ps[df_ps[treatment] == 1]

In [ ]:
# student's t-test for revenue (dependent variable)
from scipy.stats import ttest_ind

print(df_control.DI_agri_extreme_M7.mean(), df_treatment.DI_agri_extreme_M7.mean())

# compare samples
_, p = ttest_ind(df_control.DI_agri_extreme_M7, df_treatment.DI_agri_extreme_M7)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

#### Calculate logistic propensity scores.

We fit a logistic regression to our pixels with all our variables, and we are trying to predict our treatment (T). Unlike most machine learning tasks we don’t calculate the accuracy here. It is the predicted proability we are interested in.

Propensity Score is the raw predicted probability from a logistic regression (logit) model, indicating the likelihood of an unit being in the treatment group based on covariates.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

## TODO: train your logistic regression model here

print(f"Treatment Model Accuracy: {accuracy:.4f}")
print(f"Treatment Model AUC: {auc_score:.4f}")

Plotting density functions for treated and untreated groups

In [ ]:
# The predicted_data attribute contains the 'propensity_score' column
sns.kdeplot(data=df_ps[df_ps[treatment] == 1], 
            x='prop_score', label='Treated', fill=True)
sns.kdeplot(data=df_ps[df_ps[treatment] == 0], 
            x='prop_score', label='Untreated (Original)', fill=True)

plt.title('Propensity Score Density')
plt.xlabel('Propensity Score')
plt.legend()
plt.show()

### Filter out samples with extreme propensity scores

In [ ]:
# 1. Identify the IDs that fall within the common support
valid_ids = df_ps.loc[
    (df_ps["prop_score"] > 0.05) & (df_ps["prop_score"] < 0.95), "id"
]
print(np.unique(valid_ids).shape)

In [ ]:
# 2. Select rows from the original samples where the 'id' is in our valid list
# This ensures you keep every single original column.
df_trimmed = samples[samples["id"].isin(valid_ids)]
df_trimmed_plot = df_ps[df_ps["id"].isin(valid_ids)]

# Print the diagnostics
removed_count = len(samples) - len(df_trimmed)
print(f"Removed {removed_count} samples due to extreme propensity scores.")
print(f"New dataset size: {len(df_trimmed)} samples.")

# Save to a new CSV file
df_trimmed.to_csv("/content/drive/MyDrive/09_challenge_EllisWinterSchool/df_ps_trimmed.csv", index=False)

In [ ]:
# The predicted_data attribute contains the 'propensity_score' column
sns.kdeplot(data=df_trimmed_plot[df_trimmed_plot[treatment] ==1], 
            x='prop_score', label='Treated', fill=True)
sns.kdeplot(data=df_trimmed_plot[df_trimmed_plot[treatment] ==0], 
            x='prop_score', label='Untreated (Original)', fill=True)

plt.title('Propensity Score Density')
plt.xlabel('Propensity Score')
plt.legend()
plt.show()

### Check stats and the spatial representation of the trimmed sample data

In [ ]:
%load_ext autoreload
%autoreload 2
from utils.utils import *

In [ ]:
plot_split_violin_mosaic(df_trimmed, target_var="DI_agri_extreme_M7", 
                          define_features_list=inp_vars, ncols=6)

Plot the distribution of the Outcome and Treatment

In [ ]:
fig, axs = plot_spatial_temporal_grid(df_trimmed, treatment, bound_country)
plt.show()

In [ ]:
fig, axs = plot_spatial_temporal_grid(df_trimmed, "DI_agri_extreme_M7", bound_country)
plt.show()